In [1]:
import os
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
from pyspark.sql import SQLContext
from pyspark.ml.feature import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, Word2Vec, StringIndexer, IndexToString, StopWordsRemover
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit




/home/hadi/.local/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:
os.getcwd()
os.chdir('/home/hadi/Datasets')

Without Distributed Computing/Without PySpark


In [4]:
df_undistributed=pd.read_csv('IMDB Dataset.csv')
df_undsitributed_train=df_undistributed.iloc[:40000,:]
df_undsitributed_test=df_undistributed.iloc[10000:,:]


words=[',','<','>','?','br','/','.','-',';',':','(',')','[',']','{','}','!','@','#','$','%','^','&','*','_','+','=','|','\\','"',"'",'<br,','<br','<br>','<br/>','<br />','<br >','<br /']
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))
stop_words.add('.')
stop_words.add('<')
stop_words.add('>')
stop_words.add('?')
stop_words.add('br')
stop_words.add('/')
stop_words.add('.')
stop_words.add('-')
stop_words.add(';')

for i in range(len(df_undistributed['review'])):
	word_tokens = word_tokenize(df_undistributed['review'][i])
	filtered_sentence = [w for w in word_tokens if not w in stop_words and w != ',']
	filtered_sentence = []
	for w in word_tokens:
		if w not in stop_words and w != ',':
			lemmatizer.lemmatize(w)
			filtered_sentence.append(w)
	df_undistributed['review'][i] = ' '.join(filtered_sentence)
	df_undistributed['review'][i] = df_undistributed['review'][i].lower()


tokenized_reviews = [word_tokenize(review) for review in df_undistributed['review']]
word2vec_model = Word2Vec(sentences=tokenized_reviews, vector_size=10, window=5, min_count=1, workers=4)

print(word2vec_model)



def document_vector(word2vec_model, doc):
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    if len(doc) == 0:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(word2vec_model.wv[doc], axis=0)  

X = [document_vector(word2vec_model, doc) for doc in tokenized_reviews]
y = df_undistributed['sentiment']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: No need to vectorize data, as we already have numerical vectors

# Step 5: Train RandomForestClassifier
RF = RandomForestClassifier(max_depth=4)
RF.fit(X_train, y_train)

# Step 6: Predict and Evaluate
y_pred = RF.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1:', f1_score(y_test, y_pred, average='weighted'))
text='I love this movie, But the title is not good overall the movie was lovely'
text=word_tokenize(text)
text=[document_vector(word2vec_model, text)]
print(RF.predict(text))
print(RF.predict_proba(text))

text_2='I hate this movie'
text_2=word_tokenize(text_2)
text_2=[document_vector(word2vec_model, text_2)]
print(RF.predict(text_2))
print(RF.predict_proba(text_2))

text_3='The movie was very strong and powerful'
text_3=word_tokenize(text_3)
text_3=[document_vector(word2vec_model, text_3)]
print(RF.predict(text_3))
print(RF.predict_proba(text_3))




DISTRIBUTED WORD EMBEDDINGS

In [ ]:
spark=SparkSession.builder \
    .appName("Spark NLP") \
    .master("spark://172.28.111.195:4040") \
    .config("spark.driver.memory", "16G") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3") \
    .getOrCreate()


In [9]:
df=pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
df_train=df[:20000]
df_test=df[20000:]


In [11]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [12]:
print(df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [13]:
spark_df=spark.createDataFrame(df)
spark_df_train,spark_df_test=spark_df.randomSplit([0.8,0.2],seed=42)

In [14]:
type(spark_df_train)


pyspark.sql.dataframe.DataFrame

In [15]:


words=[',','<','>','?','br','/','.','-',';',':','(',')','[',']','{','}','!','@','#','$','%','^','&','*','_','+','=','|','\\','"',"'",'<br,','<br','<br>','<br/>','<br />','<br >','<br /']

tokenizer = Tokenizer(inputCol="review", outputCol="words")
remover=StopWordsRemover(inputCol="words", outputCol="filtered", stopWords=words)
word2vec = Word2Vec(vectorSize=10, windowSize=5, inputCol=tokenizer.getOutputCol(), outputCol="features")
labelIndexer = StringIndexer(inputCol="sentiment", outputCol="label").fit(spark_df_train)
rf = RandomForestClassifier(numTrees=100, maxDepth=4, labelCol="label", featuresCol="features")
labelConverter = IndexToString(inputCol="prediction", outputCol="predicted_sentiment", labels=labelIndexer.labels)



pipeline = Pipeline(stages=[tokenizer,remover, word2vec, labelIndexer, rf, labelConverter])
model = pipeline.fit(spark_df_train)
prediction = model.transform(spark_df_test)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print("Accuracy:", accuracy)




24/04/24 11:51:20 WARN TaskSetManager: Stage 0 contains a task of very large size (8078 KiB). The maximum recommended task size is 1000 KiB.
24/04/24 11:51:27 WARN TaskSetManager: Stage 3 contains a task of very large size (8078 KiB). The maximum recommended task size is 1000 KiB.
24/04/24 11:51:38 WARN TaskSetManager: Stage 5 contains a task of very large size (8078 KiB). The maximum recommended task size is 1000 KiB.
24/04/24 11:51:44 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/04/24 11:52:39 WARN TaskSetManager: Stage 8 contains a task of very large size (8078 KiB). The maximum recommended task size is 1000 KiB.
24/04/24 11:52:39 WARN TaskSetManager: Stage 9 contains a task of very large size (8078 KiB). The maximum recommended task size is 1000 KiB.
24/04/24 11:52:42 WARN TaskSetManager: Stage 10 contains a task of very large size (8078 KiB). The maximum recommended task size is 1000 KiB.
24/04/24 11:52:45 WARN TaskSetManager: Stage 1

Accuracy: 0.6237422502286818


In [16]:
evaluator = BinaryClassificationEvaluator(labelCol="label", prediction="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print("Accuracy:", accuracy)

TypeError: BinaryClassificationEvaluator.__init__() got an unexpected keyword argument 'prediction'

In [17]:
text_spark='I love this movie, But the title is not good overall the movie was lovely'
text_spark_2='I hate this movie'
text_spark_3='The movie was very strong and powerful'

text_data=[text_spark,text_spark_2,text_spark_3]
text_data=pd.DataFrame(text_data,columns=['review'])
text_data=spark.createDataFrame(text_data)

prediction=model.transform(text_data)
prediction.show()


24/04/24 11:54:26 WARN StringIndexerModel: Input column sentiment does not exist during transformation. Skip StringIndexerModel for this column.


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|              review|               words|            filtered|            features|       rawPrediction|         probability|prediction|predicted_sentiment|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|I love this movie...|[i, love, this, m...|[i, love, this, m...|[-0.3091462463140...|[45.3715566107725...|[0.45371556610772...|       1.0|           positive|
|   I hate this movie|[i, hate, this, m...|[i, hate, this, m...|[-0.5255057960748...|[54.3145256390977...|[0.54314525639097...|       0.0|           negative|
|The movie was ver...|[the, movie, was,...|[the, movie, was,...|[-0.1830796450376...|[37.9402120425197...|[0.37940212042519...|       1.0|           positive|
+--------------------+--------------------+---